## 简单学习Dataset

- Dataset是pytorch中训练模型的所有数据集的父类，所有的数据集都要继承Dataset这个类，
- 在自定义数据集的时候，需要重写`__getitem__`以及`__len__`方法
- 现在数据集是flower_photo数据集

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os

class ClassifyDataset(Dataset):
    """
    数据分类数据集
    Args:
        root_dir: 图片数据集的根目录
        transform: 可选的数据增强或预处理操作
    """
    def __init__(self, root_dir: str, transform=None):
        super().__init__()
        self.root_dir = root_dir
        self.transform = transform
        
        # 获取所有类别（文件夹名）
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        # 收集所有图片路径及其对应的类别索引
        self.image_paths = []
        self.labels = []
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            if not os.path.isdir(cls_dir):  # 确保是文件夹
                continue
            for img_name in os.listdir(cls_dir):
                img_path = os.path.join(cls_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        """返回数据集的大小"""
        return len(self.image_paths)

    def __getitem__(self, index):
        """
        根据索引获取单个样本
        Args:
            index: 样本索引
        Returns:
            image: PIL.Image 或 Tensor，图片数据
            label: int，类别索引
        """
        # 加载图片
        img_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(img_path).convert("RGB")  # 确保图片是 RGB 格式

        # 应用数据变换（如数据增强、归一化等）
        if self.transform is not None:
            image = self.transform(image)

        return image, label